In [1]:
%load_ext autoreload
%autoreload 2

# import

In [2]:
import torch
import gym
from firefly_task import Model as Task
from stable_baselines3 import PPO
from ult import *
import os
import sys
sys.path.append(f'{os.getcwd()}')
import warnings
warnings.filterwarnings("ignore")

In [3]:
from cmaes import CMA
import copy

# prepare

In [4]:
modelname = 'movingff_test0908'

epoch_size = 33333
n_epoch = 55
n_seed = 1
seed = 0
np.random.seed(seed)
torch.manual_seed(seed)

# for plotting
num_eval_episode = 2222  # for eval plot during training



In [5]:
# which agent checkpoint to use
iepoch=35

In [6]:
# load agent and task
env=Task(dt=1)
env.debug=0
thismodel = f'seed{seed}_model{modelname}_ep{iepoch}'
model = PPO.load(f'ycstore/{thismodel}')
# agent=agent_.actor.mu.cpu()


In [7]:
# create some simulation data
num_eval_episode=22222
# eval collection --------------------
results=[]
thismodel = f'seed{seed}_model{modelname}_ep{iepoch}'
model = PPO.load(f'ycstore/{thismodel}')

def eval_wrapper(a):
    episode = run_one_episode(env, model, deterministic=True)
    return episode

with multiprocess.Pool(processes=8) as pool:
    all_episode_data = pool.map(eval_wrapper, range(num_eval_episode))

for episode in all_episode_data:
    # process
    episode['sum_reward']=sum(episode['rewards'])
    results.append(episode)

df = pd.DataFrame(results)
print(f'avg reward per eval trials: {sum(np.concatenate(df.rewards.to_numpy()))/num_eval_episode:.2f}')

avg reward per eval trials: 0.65


In [ ]:
# # create the dataset from real data
# path=Path('data/')
# datafiles=list(path.rglob('packed'))
# states, actions, tasks=[],[],[]
# for file in datafiles:
#     with open(file, 'rb') as f:
#         df= pickle.load(f)
#     df=datawash(df)
#     df=df[df.category=='normal']
#     df=df[df.target_r>250]
#     s, a, t=monkey_data_downsampled(df,factor=0.0025)
#     states+=s
#     actions+=a
#     tasks+=t
# del df


In [ ]:
# continue on checkpoint or start fresh.
resfile=Path(f'yctore/movingff_inverse/{modelname}_testinv')
optimizer=None
log=[]
if resfile.is_file():
    print('continue on previous inverse...')
    with open(resfile, 'rb') as f:
        log = pickle.load(f)
    optimizer=log[-1][0]
else:
    print('starting new inverse ...')
    
def getlogll(x):
    with torch.no_grad():
        return  monkeyloss_(agent, actions, tasks, phi, torch.tensor(x).t(), env, action_var=1e-3,num_iteration=1, states=states, samples=5,gpu=False).item()

# fresh start
if not optimizer:
    # init condition, we want to at least cover some dynamic range
    init_theta=torch.tensor([[0.5],   
            [1.0],   
            [0.5],   
            [0.5],   
            [0.5],   
            [0.5],   
            [0.13],   
            [0.5],   
            [0.5],   
            [0.5],   
            [0.5]])
    dim=init_theta.shape[0]
    init_cov=torch.diag(torch.ones(dim))*0.3
    cur_mu=init_theta.view(-1)
    cur_cov=init_cov
    optimizer = CMA(mean=np.array(cur_mu), sigma=0.5,population_size=14)
    optimizer.set_bounds(np.array([
    [0.2, 0.7, 0.01, 0.01, 0.01, 0.01, 0.129, 0.1, 0.1, 0.1, 0.1],
    [1.,2,1,1,1,1,0.131,0.9, 0.9, 0.9, 0.9]],dtype='float32').transpose())


In [ ]:

for generation in range(len(log),len(log)+399):
    start=timer()
    solutions = []
    xs=[]
    for _ in range(optimizer.population_size):
        x = optimizer.ask().astype('float32')
        xs.append(x)
    solutions=ray.get([getlogll.remote(p) for p in xs])
    meanlogll=np.mean(solutions)
    solutions=[[x,s] for x,s in zip(xs,solutions)]
    optimizer.tell(solutions)
    log.append([copy.deepcopy(optimizer), xs, solutions])

    with open(resfile, 'wb') as handle:
        pickle.dump(log, handle, protocol=pickle.HIGHEST_PROTOCOL)
    print('done, ',timer()-start)
    print("generation: ",generation,'-logll: ',meanlogll)
    print('cur estimatation ',["{0:0.2f}".format(i) for i in optimizer._mean])
    print('cur uncertainty ',["{0:0.2f}".format(i) for i in np.diag(optimizer._C)**0.5])
    notify(msg="".join(['{:0.1f} '.format(i) for i in optimizer._mean]))
    if optimizer.should_stop():
        print('stop at {}th generation'.format(str(generation)))




    